In [1]:
import zombie_imp
import csv
import serial
import time 
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
# Initialize the Serial port
ser = serial.Serial('COM8', 115200)  # Change the port and baud rate according to transmitter

# create the CSV file
with open('sensor_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Cell 1', 'Cell 2', 'Cell 3'])  

    # read the data from port and store in CSV file
    try:
        while True:
            data = ser.readline().decode('utf-8').strip()  # read data from port
            if data:
                values = data.split(' ')  # split with space
                if len(values) == 3:  # make sure there are three values
                    writer.writerow([int(val) for val in values])  # store the data
    except KeyboardInterrupt:  # stop iteration by Ctrl+C
        pass